In [7]:
import requests
import json

# Define the API URL
url = "http://127.0.0.1:5000/predict"

# Example input data (must match your model's expected dimensions)
data = {
    "features": [
        [0.1]*128,  # 128-dim feature vector
        [0.2]*128   # another node
    ],
    "edge_index": [
        [0, 1],  # source nodes
        [1, 0]   # destination nodes
    ]
}

# Send POST request
response = requests.post(url, json=data)

# Print prediction
print("Prediction:", response.json())


Prediction: {'prediction': [1, 1]}


In [8]:
# Print the prediction labels
for i, p in enumerate(response.json()["prediction"]):
    label = "THREAT" if p == 1 else "BENIGN"
    print(f"Node {i}: {label}")

Node 0: THREAT
Node 1: THREAT


In [ ]:
import requests
import time
import torch
from gnn_model import ThreatDetectionGNN

# Load GNN Model
model = ThreatDetectionGNN(input_dim=128, hidden_dim=64, output_dim=2)
model.load_state_dict(torch.load("gnn_model.pth"))
model.eval()

# Elasticsearch credentials
es_url = "http://localhost:9200/winlogbeat-*/_search"
auth = ("elastic", "2F8u4KWbdpHG24abCZkd")

# Last timestamp processed
last_ts = "now-1m"

while True:
    # Step 1: Get new logs
    query = {
        "size": 10,
        "sort": [{ "@timestamp": "asc" }],
        "query": {
            "range": {
                "@timestamp": { "gt": last_ts }
            }
        }
    }

    res = requests.get(es_url, auth=auth, json=query)
    hits = res.json().get("hits", {}).get("hits", [])

    for hit in hits:
        log = hit["_source"]

        # Step 2: Extract features (Dummy vector here; replace with actual extraction)
        features = [[0.1] * 128]  # Replace with real vector from `log`
        edge_index = [[0], [0]]  # Dummy edge index

        # Step 3: Run prediction
        with torch.no_grad():
            x = torch.tensor(features).float()
            ei = torch.tensor(edge_index).long()
            output = model(x, ei)
            prediction = torch.argmax(output, dim=1).item()

        # Step 4: Index result into Elasticsearch
        detection = {
            "@timestamp": log["@timestamp"],
            "event_id": log.get("event", {}).get("code", ""),
            "username": log.get("user", {}).get("name", "unknown"),
            "source_ip": log.get("host", {}).get("ip", [""])[-1],
            "prediction": prediction,
            "threat_level": "high" if prediction == 1 else "low"
        }

        requests.post(
            "http://localhost:9200/gnn-threat-detection/_doc",
            auth=auth,
            json=detection
        )

        last_ts = log["@timestamp"]

    print("Checked new logs. Sleeping...")
    time.sleep(10)


Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked new logs. Sleeping...
Checked ne

In [ ]:
import requests
import json
url = "http://127.0.0.1:5000/predict"
# Load exported logs
with open("logs.json", "r") as file:
    logs = json.load(file)

# Prepare the request
data = {
    "features": logs["hits"]["hits"][:100]  # Send 100 logs for processing
}

# Send request to the Flask API
response = requests.post(url, json=data)
print("Prediction:", response.json())